## Neural networks
### 1. A simple example of data transformation

Neural networks are a good tool for both classification because they can automatically perform a non-linear change of basis on the data that is optimal for prediction.  They also have the capacity to give us some really useful insights into how they work and why they're so good at generalizing.  Here, we'll construct one using the library [Keras](https://keras.io/), which is a neural network API for [Tensorflow](https://www.tensorflow.org/).  Keras makes the construction of neural networks very straightforward and intuitive.

We can begin with out normal imports:

In [ ]:
from __future__ import division,print_function

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize']= (16,9)

We'll be generating initial guesses, etc. using random numbers.  To ensure that things go according to plan, let's seed the numpy pseudorandom number generator

In [ ]:
np.random.seed(0)

Now let's make some data to classify.  We can choose any function, but if we want to challenge this thing, it should be something that would fail under normal logistic regression.  For example, let's generate some data that is Bernoulli distributed with $\theta(x)$ given by a bell curve.   I'll generate data from this distribution using a variant of [rejection sampling](https://en.wikipedia.org/wiki/Rejection_sampling).

In [ ]:
classes = [0,1]

m_train = 500
m_test = 250
X_train = np.random.rand(m_train)
X_test = np.random.rand(m_test)

X_train.sort()
X_test.sort()


theta = np.exp(-((X_train-0.5)/0.2)**2)# + np.exp(-((X-0.75)/0.1)**2)
theta /= theta.max()
a = np.random.rand(m_train)
y_train = (a<=theta).astype(int)

theta = np.exp(-((X_test-0.5)/0.2)**2)# + np.exp(-((X-0.75)/0.1)**2)
theta /= theta.max()
a = np.random.rand(m_test)
y_test = (a<=theta).astype(int)

import keras.utils

y_train = keras.utils.to_categorical(y_train, 2) 
y_test = keras.utils.to_categorical(y_test, 2)
plt.plot(X_train,y_train[:,1],'k.')
plt.plot(X_test,y_test[:,1],'r.')
plt.show()

This data is not linearly-seperable (not even roughly).  Rather, it is multimodal:  Naive Bayes would be bound to fail because we could not *a priori* determine a sensible probability model for the data.  Logistic regression with the linear basis would also be bound to fail because it can't deal with multiple peaks like this (although we could enrich the basis set instead).  A neural network will allow us to *learn* good basis functions, or how to transform the data to optimize classification.    

Let's first try making a logistic (or rather, Softmax) regression using Keras.  We can then enhance this basic classifier into a true neural network.  We will need a few imports: Keras itself, and then for convenience its models and layers modules.  Models contains classes that represent the structure of the neural network in general (i.e. is it a feed-forward network, or is it recurrent) and holds layers, which are collections of nodes (inputs->activation function->output).

In [ ]:
import keras 
import keras.models as km
import keras.layers as kl

The graph for logistic regression is feed-forward, or sequential, which implies that inputs flow from input to output without doubling back: there are no connections from layers to previous layers.  We can instantiate such a model as follows: 

In [ ]:
logistic_model = km.Sequential()

Now we can add a layer.  Our problem is univariate, which means that we will only input one feature.  However, we need to output two classes.  Our activation function will be softmax, and we want to add a bias node to the input.  Such a layer can be generated as follows.

In [ ]:
output_layer = kl.Dense(2,input_shape=(1,),activation='softmax',use_bias=True)
logistic_model.add(output_layer)

Simple as that for constructing the network.  Now, we can classify our inputs as follows:

In [ ]:
logistic_model.predict(X_test[:5])

which gives us the probability of class membership for each class.  However, we have yet to do an important step: training!  To train, we need to define a loss function (the thing to minimize) and an optimizer (how to do the minimization).  We can then compile our model using these loss functions.

In [ ]:
# Cross entropy
loss = keras.losses.categorical_crossentropy

# Stochastic gradient descent
optimizer = keras.optimizers.SGD(lr=0.01)

logistic_model.compile(loss=loss,
              optimizer=optimizer,
              metrics=['accuracy'] )

Note that we've also set a *metric* which is just going to keep track of our classification accuracy, but will not be used for training purposes.  

Now we can fit our logistic regression model using the *fit* command (which is nicely similar to the sklearn api).  We will run 1000 iterations of SGD, with a batch size that is equal to the entire dataset.  As such, this isn't really stochastic gradient descent at all, but rather just normal gradient descent.  We will also ask the fit command to evaluate our loss and classification accuracy on the test set as well, in order to look at test set convergence.

In [ ]:
logistic_model.fit(X_train,y_train,batch_size=m_train,epochs=1000,verbose=1,validation_data=(X_test,y_test))

After taking a few moments to train, we can plot the predictions:

In [ ]:
y_pred = logistic_model.predict(X_test)
plt.plot(X_test,y_pred[:,1],'r.')
plt.plot(X_test,y_test[:,1],'k.')
plt.show()
y_pred

Obviously, this did a terrible job.  Our model has insufficient freedom to classify these results, so it has returned the result that there is a 2/3 chance of any point being class 0, and a 1/3 chance of it being class 1, which is the prior probabilities of the classes!  Doing logistic regression didn't help at all!

Let's see if we can make a better classifier.  To do this, we'll insert a hidden layer between the input and the output.  This hidden layer will have two nodes, both with sigmoidal activation functions.  Sort of like in polynomial regression, these hidden layers will project our univariate dataset to a higher dimensional space, where hopefully there is a better defined decision boundary between the two classes.  

It's straightforward to add this hidden layer in Keras.

In [ ]:
# Define a sequential model as before
nn_model = km.Sequential()

# How many node in the hidden layer do we want?
n_hidden = 2

# Define the hidden layer and the output softmax layer
hidden_layer = kl.Dense(n_hidden,input_shape=(1,),use_bias=True,activation='sigmoid')
softmax_layer = kl.Dense(2,use_bias=True,activation='softmax')

# Add the layers to the sequential model (Note that order matters here!)
nn_model.add(hidden_layer)
nn_model.add(softmax_layer)

nn_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.RMSprop(lr=0.01),
              metrics=['accuracy'])


convergence_history = nn_model.fit(X_train,y_train,batch_size=m_train,epochs=1000,verbose=0,validation_data=(X_test,y_test))



Let's see whether we converged by looking at a plot of training and test set loss over the course of the training.

In [ ]:
f = convergence_history.history
plt.plot(f['loss'],'k-')
plt.plot(f['val_loss'],'r--')
plt.show()

Perhaps more informative is the classification accuracy:

In [ ]:
plt.plot(f['acc'],'k-')
plt.plot(f['val_acc'],'r--')
plt.show()

Around 85% test accuracy, much better than standard logistic regression.  Let's look at the class 1 probability:

In [ ]:
y_pred = nn_model.predict(X_test)
plt.plot(X_test,y_pred[:,1],'r.')
plt.plot(X_test,y_test[:,1],'k.')
plt.show()

The decision boundary is where the red dots are exactly 0.5.  

To get a little bit better sense of what our neural network is doing, let's compute the output of the hidden layer: this is the dataset after transformation.

In [ ]:
from keras import backend as K

# with a Sequential model
get_hidden_layer_output = K.function([hidden_layer.input],
                                     [hidden_layer.output])
layer_output = get_hidden_layer_output([X_test.reshape((m_test,1))])[0]

We can easily plot this 2D dataset.

In [ ]:
plt.scatter(layer_output[:,0],layer_output[:,1],c=y_test[:,1],marker='o',alpha=0.2)
plt.show()

Is there a line that we can draw that roughly separates these classes?  Absolutely!  Something like

In [ ]:
plt.scatter(layer_output[:,0],layer_output[:,1],c=y_test[:,1],marker='o',alpha=0.2)
plt.plot(layer_output[:,0],-1.8*layer_output[:,0] + 0.6)
plt.xlim(0,1)
plt.ylim(0,1)


plt.show()

works well.  In fact, if we multiply these observations with the weights and add the biases for the softmax layer, we can also look at the log-probabilities (which are the inputs to softmax).

In [ ]:
W_2,b_2 = softmax_layer.get_weights()
final_log_probs = layer_output @ W_2 + b_2


Now, if we can plot these log-probabilities as a function of the input values of $x$ to see where our network created decision boundaries

In [ ]:
plt.plot(X_test,final_log_probs[:,0])
plt.plot(X_test,final_log_probs[:,1])
plt.scatter(X_test,np.zeros_like(X_test),c=y_pred[:,1]>y_pred[:,0])
plt.show()
print(w)
print(b)

Obviously, the decision boundaries occur where the log-probabilities are equal!

To recap, we managed to improve our classification scheme by projecting our simple 1D dataset into higher dimensions, with that transformation occurring adaptively, in the hopes that the higher dimensional dataset had a decision boundary that was more suitable than the initial data.  As it turns out, it did.  Of course this procedure can also work for much higher dimensionality, and for much more complicated cases than this one.  

### 2. Neural networks applied to MNIST
Let's apply our neural network to MNIST.  Mirroring the last section, we'll begin by programming a softmax classifier, then upgrade to something more interesting.

To begin, let's import keras, as well as the MNIST dataset

In [ ]:
import keras

from keras.datasets import mnist

import keras.models as km
import keras.layers as kl
from keras import backend as K

rows,cols = 28,28
n = rows*cols

(X_train,y_train),(X_test,y_test) = mnist.load_data()

m_train = X_train.shape[0]
m_test = X_test.shape[0]

X_train = X_train.reshape((m_train,rows*cols))
X_test = X_test.reshape((m_test,rows*cols))

X_train = X_train/255
X_test = X_test/255

y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

Note that in the above, we've rescaled the test and training data to the [0,1] interval for numerical stability.  In general, big numbers are bad.  

Next, let's define our model.  We'll use a sequential (aka feedforward) model with a single softmax output layer.  Note the presence of a regularizer on the softmax layer.  This will add a term proportional to the L2 norm on this layer's weights to the loss function.  Just like in polynomial regression, this will favor simple models and keep the network from overfitting.

In [ ]:
N = 10

model = km.Sequential()

softmax_layer = kl.Dense(N,input_shape=(n,),activation='softmax',kernel_regularizer=keras.regularizers.l2(0.001))
model.add(softmax_layer)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.RMSprop(lr=0.001),
              metrics=['accuracy'])

We will fit the model as before.  However, we will select a batch size which is *smaller* than our training data size.  This implies that we will be using *stochastic gradient descent*, in which we update our weights based on only a random subset of our data.  An epoch corresponds to having looked at every piece of data exactly once.  See the next notebook (16_stochastic_gradient_descent.ipynb) for more detail on this.

In [ ]:
batch_size = 256
epochs = 24

history = model.fit(X_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(X_test,y_test))

Training the model takes a few moments.  Let's look at our convergence properties.

In [ ]:
plt.plot(history.history['acc'],'k-')
plt.plot(history.history['val_acc'],'r--')

This model converges super fast (basically after the first few epochs)!  However, accuracy is only around 92%.  Nonetheless, it's instructive to look at the weights that are being produced here.  The inputs are being multiplied by these weights to get log probabilities, and thus if an input example has high values in the same place that there is a large weight for a given class, it is likely that the log-probability for that class will be high.  Conversely, if an input example has a high value where the weight is negative, that will contribute to a low probability.

In [ ]:
weights = softmax_layer.get_weights()
fig,axs = plt.subplots(nrows=1,ncols=10)
fig.set_size_inches(16,2)
for w,ax in zip(weights[0].T,axs):
    ax.imshow(w.reshape((28,28)),vmin=-0.25,vmax=0.25)
plt.show()

These weights are visually similar to digits.  The model is directly finding combinations of pixels that correspond to a 'one' (for example), and producing strong responses whenever those pixels are 'lit up' in a data example.

However, as in the synthetic example above, we're somewhat limited by choosing linear combinations of our pixel values as a means of classification.  What if we add the non-linear coordinate transformation of a hidden layer?  This is, again, easy to do in Keras.

In [ ]:
model = km.Sequential()

n_hidden = 256

hidden_layer = kl.Dense(n_hidden,input_shape=(n,),activation='sigmoid',kernel_regularizer=keras.regularizers.l2(0.0005))
softmax_layer = kl.Dense(N,activation='softmax')

model.add(hidden_layer)
model.add(softmax_layer)

batch_size = 256

epochs = 24

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])

history = model.fit(X_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(X_test,y_test))



We've managed to eke out an additional 4% worth of accuracy!

In [ ]:
plt.plot(history.history['acc'],'k-')
plt.plot(history.history['val_acc'],'r--')
plt.show()

As before, we can have a look at the weights produced by training the model.  These represent the combination of features that yield a strong reaction, although this time in the hidden layer, rather than in the output softmax.  In some sense, the hidden layer is extracting features from the input, then recombining them into a new representation of the data that is better suited for softmax classification.  Let's take a look at these input features.

In [ ]:
weights = hidden_layer.get_weights()
fig,axs = plt.subplots(nrows=1,ncols=10)
fig.set_size_inches(16,2)
for w,ax in zip(weights[0].T[:10],axs):
    ax.imshow(w.reshape((28,28)))
plt.show()

These are much more abstract!  By adding the hidden layer, we've managed to increase our accuracy a lot, but what we've lost is the capacity to really understand intuitively what the neural network is doing.  This is a key point that separates so-called 'deep learning' from statistical or inverse modelling.  Also, we can't do this same process for the weights of subsequent layers because they are not amenable to reshaping as images, so it will remain difficult to understand the output of the 300D hidden layer.

### 3. Deep Networks
However, if we're going to go down this path, it's reasonable to go down it even further.  Nothing stops us from adding more than one hidden layer, and perhaps this will allow us to produce even better classification accuracy.  Let's try another with two hidden layers.  However, before we do that, let's look at a couple of neural-network specific advances that make these so-called *deep* neural networks work a little bit better.

#### 3.1 Dropout
Dropout is the dominant form of regularization when training complicated neural networks.  It differs conceptually from L2 regularization in that it does not work by adding an extra term to the loss function.  Rather, every time we evaluate the model (and its gradient) during the training phase, we drop a random selection of the nodes in the layer.  This means that particular neurons can never be relied upon to produce an accurate classification, and thus it promotes the development of networks that are less prone to overfitting.  The [original paper](http://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf) is surprisingly readable, and their figure of a dropout network is included below.  Another way of understanding dropout is in the context of model averaging: produce N random neural networks that all fit the data well, then when making a prediction take the average of these. 

<img src="images/dropout.png">

#### 3.4 Rectified Linear Units
So far, we have motivated neural networks as being 'stacks' of logistic regressions, with the activation functions being sigmoids (or perhaps softmax for the classification problem).  However, while softmax will always be useful for the output layer, we can make some somewhat more exotic choices for the hidden layer activations.  By far the most popular choice in modern neural networks is the so-called *relu* activation.  It is defined as 
$$
\mathrm{ReLU}(x) = \begin{cases} x & x>0 \\
                                 0 & \mathrm{else} \end{cases}
$$                              
and looks like:

In [ ]:
x = np.linspace(-1,1,101)
relu_x = np.copy(x)
relu_x[x<0] = 0

# Sigmoid with arbitrary scaling for comparison
sigmoid_x = 1./(1+np.exp(-6*x))

plt.plot(x,relu_x)
plt.plot(x,sigmoid_x,'r--')

plt.show()

Quite different from the sigmoid!  Also, we've left behind the pesky biological motivation for activation functions as neurons: in no sense does a ReLU imitate a brain cell.  Nonetheless, it has shown to work very well for neural networks because it never gets too flat (as long as you're on the right side of the graph), and so the gradient never gets terribly small.

As an aside, there are many other activation functions that can be used.  See the following chart, copied from rasbt.github.io.
<img src="images/activations.png">

In [ ]:
model = km.Sequential()

n_hidden_1 = 256
n_hidden_2 = 256

hidden_layer_1 = kl.Dense(n_hidden_1,input_shape=(n,),activation='relu',kernel_regularizer=keras.regularizers.l2(0.001))
dropout_1 = kl.Dropout(0.2)
hidden_layer_2 = kl.Dense(n_hidden_2,activation='relu')
dropout_2 = kl.Dropout(0.2)
softmax_layer = kl.Dense(N,activation='softmax')

layers = [hidden_layer_1, dropout_1, hidden_layer_2, dropout_2, softmax_layer]
for l in layers:
    model.add(l)

batch_size = 256

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])

epochs = 12
model.fit(X_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(X_test,y_test))
score = model.evaluate(X_test,y_test,verbose=1)

We are now achieving around 98% accuracy on both the training and the test sets, at the cost of increased model complexity, inferior scrutability, and longer training times.  We can still look at the first layer to see the features that the model is extracting

In [ ]:
weights = hidden_layer_1.get_weights()
fig,axs = plt.subplots(nrows=1,ncols=10)
fig.set_size_inches(16,2)
for w,ax in zip(weights[0].T[:10],axs):
    ax.imshow(w.reshape((28,28)))
plt.show()

But they're pretty weird, and who knows what's going on in the hidden layers.  Still, can't argue with results!

Is this the best that we can do?  As it turns out, it is not.  Next time, we will discuss a special type of neural network that is specially geared towards structured data like images, sound, and sequences called a convolutional neural network.  In doing so, we will see that we can boost accuracy on this dataset to nearly 100%, and will also learn the underlying mechanisms used in computer image classification.